In [14]:
# Chemin vers le fichier texte
file_path = "/content/communes_text/Afa_Corse-du-Sud.txt"
# Fonction pour lire le contenu d'un fichier texte
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Fonction pour diviser le texte en 20 chunks
def split_into_chunks(text, num_chunks=20):
    tokens = tokenizer.encode(text)  # Tokeniser le texte
    chunk_size = len(tokens) // num_chunks  # Taille de chaque chunk
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Si le dernier chunk est trop petit, le réajuster
    if len(chunks) > num_chunks:
        chunks = chunks[:num_chunks]

    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]  # Convertir tokens en texte
    return chunk_texts

# Lire le contenu du fichier
full_text = read_file(file_path)

# Diviser le texte en 20 chunks
chunks = split_into_chunks(full_text, num_chunks=20)

# Afficher chaque chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{len(chunk)}\n")
    print("-" * 80)

Chunk 1:
298

--------------------------------------------------------------------------------
Chunk 2:
374

--------------------------------------------------------------------------------
Chunk 3:
304

--------------------------------------------------------------------------------
Chunk 4:
309

--------------------------------------------------------------------------------
Chunk 5:
272

--------------------------------------------------------------------------------
Chunk 6:
312

--------------------------------------------------------------------------------
Chunk 7:
263

--------------------------------------------------------------------------------
Chunk 8:
217

--------------------------------------------------------------------------------
Chunk 9:
221

--------------------------------------------------------------------------------
Chunk 10:
190

--------------------------------------------------------------------------------
Chunk 11:
290

----------------------------------

In [4]:
%%capture
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers
!pip install keybert ctransformers[cuda]
!pip install --upgrade git+https://github.com/huggingface/transformers

In [1]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoTokenizer, pipeline

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
prompt = """
I have the following document:
* The website mentions that it only takes a couple of days to deliver but I still have not received mine

Extract 5 keywords from that document.
"""
response = generator(prompt)
print(response[0]["generated_text"])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA_gather)

In [2]:
import os
import time
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Charger le modèle et le tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Créer un pipeline de génération de texte avec GPU
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=2500,
    repetition_penalty=1.1,
    device=0  # Utiliser le GPU
)

# Chemin vers le dossier contenant les fichiers texte
input_dir = "communes_text"

# Fonction pour lire le contenu d'un fichier texte
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Fonction pour diviser le texte en 20 chunks
def split_into_chunks(text, num_chunks=20):
    tokens = tokenizer.encode(text)  # Encoder le texte
    chunk_size = len(tokens) // num_chunks  # Taille de chaque chunk
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Si le dernier chunk est trop petit, ajuster
    if len(chunks) > num_chunks:
        chunks = chunks[:num_chunks]

    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]  # Convertir tokens en texte
    return chunk_texts

# Boucle pour traiter chaque fichier dans le dossier
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(input_dir, filename)
        full_text = read_file(file_path)

        # Diviser le texte en 20 chunks
        chunks = split_into_chunks(full_text, num_chunks=20)

        # Générer les prompts et obtenir les réponses
        for i, chunk in enumerate(chunks):
            prompt = f"""
I have the following document:
{chunk}

Indicate whether this document describes history, economy, or other. By history, we mean past events, historical figures, battles, etc. By economy, we mean economic aspects like industries, finances, trade, etc. If it does not fit these categories, indicate other.
"""
            response = generator(prompt)
            print(response[0]["generated_text"])

        # Pause pour éviter les limites de taux
        time.sleep(1)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA_gather)

# 📄 **Prompt Engineering**


# 📄 **Zero-Shot Prompting**


In [7]:
prompt = """
I have the following document:
*  The city is, with Bastia, the economic, commercial and administrative centre of Corsica. Its urban area of nearly 90,000 inhabitants is spread over a large part of the Corse-du-Sud, on either side of the Gulf of Ajaccio and up the valley of the Gravona. Its business is primarily oriented towards the services sector. The services sector is by far the main source of employment in the city. Ajaccio is an administrative centre comprising communal, intercommunal, departmental, regional, and prefectural services. It is also a shopping centre with the commercial streets of the city centre and the areas of peripheral activities such as that of Mezzavia (hypermarket Géant Casino) and along the ring road (hypermarket Carrefour and E. Leclerc). Tourism is one of the most vital aspects of the economy, split between the seaside tourism of summer, cultural tourism, and fishing. A number of hotels, varying from one star to five star, are present across the commune. Ajaccio is the seat of the Chamber of Commerce and Industry of Ajaccio and Corsica South. It manages the ports of Ajaccio, Bonifacio, Porto-Vecchio, Propriano and the Tino Rossi marina. It also manages Ajaccio airport and Figari airport as well as the convention centre and the Centre of Ricanto. Secondary industry is underdeveloped, apart from the aeronautical company Corsica Aerospace Composites CCA, the largest company on the island with 135 employees at two sites. ,

Indicate whether this document describes history, economy, or other. By history, we mean past events, historical figures, battles, etc. By economy, we mean economic aspects like industries, finances, trade, etc. If it does not fit these categories, indicate other.
"""
response = generator(prompt)
print(response[0]["generated_text"])



I have the following document:
*  The city is, with Bastia, the economic, commercial and administrative centre of Corsica. Its urban area of nearly 90,000 inhabitants is spread over a large part of the Corse-du-Sud, on either side of the Gulf of Ajaccio and up the valley of the Gravona. Its business is primarily oriented towards the services sector. The services sector is by far the main source of employment in the city. Ajaccio is an administrative centre comprising communal, intercommunal, departmental, regional, and prefectural services. It is also a shopping centre with the commercial streets of the city centre and the areas of peripheral activities such as that of Mezzavia (hypermarket Géant Casino) and along the ring road (hypermarket Carrefour and E. Leclerc). Tourism is one of the most vital aspects of the economy, split between the seaside tourism of summer, cultural tourism, and fishing. A number of hotels, varying from one star to five star, are present across the commune. 

# 📄 **Keyword Prompting**
